In [122]:
import xml.etree.cElementTree as et
import pandas as pd
import os
import warnings
from lxml import etree
import pyodbc
import pyvalidata as pvd
import hashlib
import ibm_db_dbi as idb
import time
import datetime
import pyvalidata as pvd
from pathlib import Path
import datetime as dt
from dotenv import load_dotenv

In [123]:
load_dotenv(verbose=True)

True

In [124]:
warnings.filterwarnings(action="ignore", category=UserWarning)
warnings.filterwarnings(action="ignore", category=FutureWarning)
warnings.filterwarnings(action="ignore", category=DeprecationWarning)


# Input

In [125]:
folder_path=r'S:\iss\QA\QACOE_PythonProjects\Xml_validation\20230331'

# Database Connections

In [126]:
# KCMS test

kcms_hostname = "u060dwd21"
kcms_database = "UDBINT"
kcms_port = "50000"
kcms_protocol = "TCPIP"
kcms_username = os.environ.get('kcms_UID')
kcms_password = os.environ.get('kcms_PWD')
kcms_conn_str = "DATABASE=" f"{kcms_database}" ";" "HOSTNAME=" f"{kcms_hostname}" ";" "PORT=" f"{kcms_port}" ";" "PROTOCOL=" f"{kcms_protocol}" ";" "UID=" f"{kcms_username}" ";" "PWD=" f"{kcms_password}" ";"
kcmst_conn = idb.connect(kcms_conn_str, "", "")
print("\n\nConnected to KCMS Test Database.")



Connected to KCMS Test Database.


In [127]:
# KODS Prod

kods_hostname = "u060odsd101.kroger.com"
kods_database = "ODSPROD"
kods_port = "50000"
kods_protocol = "TCPIP"
kods_username = os.environ.get('kods_UID')
kods_password = os.environ.get('kods_PWD')
kods_conn_str = "DATABASE=" f"{kods_database}" ";" "HOSTNAME=" f"{kods_hostname}" ";" "PORT=" f"{kods_port}" ";" "PROTOCOL=" f"{kods_protocol}" ";" "UID=" f"{kods_username}" ";" "PWD=" f"{kods_password}" ";"
kods_conn = idb.connect(kods_conn_str, "", "")
print("\n\nConnected to KODS Database.")



Connected to KODS Database.


In [128]:
def read_large_sql(query, conn, n):
    dfs = []
    for chunk in pd.read_sql_query(query, con=conn, chunksize=n):
        dfs.append(chunk)
    return pd.concat(dfs)

In [129]:
def get_order_group(xmlfiles):    
    ordergroupdict={}
    for xmlfile in xmlfiles:
        xml_file_path = os.path.join(folder_path,xmlfile)
        tree=et.parse(xml_file_path)
        root=tree.getroot()

#         print(f"Processing file :{xmlfile}") 
        for l1, Transaction in enumerate(root.findall('Transaction')): 
                ref_count = 0
                sale_count = 0
                root1=Transaction
                seq = Transaction.find('SequenceNumber').text
                for l2, ordno in enumerate(root.iter('OrderNumber')):
                    ordno = ordno.text
#                 if ordergroupdict.get(ordno,'x')=='x':                
#                     ordergroupdict[ordno]={}
#                     elist=[]
#                     ordergroupdict[ordno]['refundxml']=elist
#                 for l3, LineItem in enumerate(root.iter('LineItem')):
#                     root1=LineItem
#                     if(root1.findall('Sale')):
#                         for l5, sale in enumerate(root1.findall('Sale')):

#                             root2=sale
#                             if(root2.findall('POSIdentity')):
#                                 for l6, pos in enumerate(root2.findall('POSIdentity')):
#                                     posItem = pos.attrib['POSIDType']#.text
#                                     if posItem=='00':
#                                         sale_count+=1
#                     if(root1.findall('Return')):
#                         for l6, ret in enumerate(root1.findall('Return')):

#                             root2=ret
#                             if(root2.findall('POSIdentity')):
#                                 for l6, rpos in enumerate(root2.findall('POSIdentity')):
#                                     rposItem = rpos.attrib['POSIDType']#.text
#                                     if rposItem=='20':
#                                         ref_count+=1
                
#                 if sale_count>0:                
#                     ordergroupdict[ordno]['salesxml']=xmlfile
#                 elif ref_count>0:
#                     tmp=ordergroupdict[ordno]['refundxml']
#                     tmp.append(xmlfile)
#                     ordergroupdict[ordno]['refundxml']=tmp
                ordergroupdict[ordno]=xmlfile
    
    return ordergroupdict



In [130]:
def remove_namespace(doc, namespace):
        """Remove namespace in the passed document in place."""
        ns = u'{%s}' % namespace
        nsl = len(ns)
        for elem in doc.getiterator():
            if elem.tag.startswith(ns):
                elem.tag = elem.tag[nsl:]


In [131]:
def processesing(file_name,folder_path):
    xml_file = os.path.join(folder_path,file_name)
    new_path = os.path.join(folder_path,'xmldata')
    if os.path.exists(new_path):
        os.mkdir(new_path)
    new_xml_file = os.path.join(new_path,xml_file)
    
    outputfilename = os.path.join(folder_path,f'{file_name[:-3]}csv')

    tree=et.parse(xml_file)
    root=tree.getroot()

    namespace_txt=str(root.tag)[1:str(root.tag).rfind("}")]    
    remove_namespace(root, namespace_txt)
    tree.write(new_xml_file, xml_declaration=True, encoding='UTF-8')
    return new_xml_file

In [132]:
def parse_xml_to_df(root):
    df =pd.DataFrame(columns=['UnitID', 'SequenceNumber', 'ReceiptDateTime', 'WorkstationID','POSItemID', 'Quantity','UnitAmount', 'ExtendedAmount','SubtractAmt','NetQuantity','NetAmount','Desc'])
    for l1, Transaction in enumerate(root.findall('Transaction')):
        if l1==0:
            rw=l1
        else:
            rw=len(df)
        root1=Transaction
        seq = Transaction.find('SequenceNumber').text
        recDt = Transaction.find('ReceiptDateTime').text
        ws = Transaction.find('WorkstationID').text

        for l2, bu in enumerate(root.iter('UnitID')):
            bu = bu.text

        for l3, LineItem in enumerate(root.iter('LineItem')):
            root1=LineItem
            if l3!=0:
                rw = len(df)
            if(root1.findall('Sale')):
                for l5, sale in enumerate(root1.findall('Sale')):
                    Qnt = sale.find('Quantity').text
                    uAmt = sale.find('UnitRetailPrice').text
                    ExAmt = sale.find('ExtendedAmount').text
                    df.at[rw,'Quantity']=Qnt
                    df.at[rw,'UnitAmount']=uAmt
                    df.at[rw,'ExtendedAmount']=ExAmt
                    df.at[rw,'NetQuantity']=float(Qnt) if (Qnt==Qnt) else Qnt
                    
                    root2=sale
                    if(root2.findall('POSIdentity')):
                        for l6, pos in enumerate(root2.findall('POSIdentity')):
                            posItem = pos.find('POSItemID').text
                            df.at[rw,'POSItemID']=posItem
                    root2=sale
                    subamt = 0
                    if(root2.findall('RetailPriceModifier')):
                        subamt=0
                        for l7, am in enumerate(root2.findall('RetailPriceModifier')):
                            
                            subamt +=float(am.find('Amount').text)
                        df.at[rw,'SubtractAmt']=subamt
                    df.at[rw,'NetAmount']=float(ExAmt)-float(subamt) if (ExAmt==ExAmt and subamt==subamt) else float(ExAmt)
            df.at[rw,'UnitID']=bu
            df.at[rw,'SequenceNumber']=seq
            df.at[rw,'ReceiptDateTime']=recDt
            df.at[rw,'WorkstationID']=ws
    
    return df

In [133]:
def parse_refundxml_to_df(root):
    df = pd.DataFrame(columns=['UnitID', 'RefundSequenceNumber', 'RefundReceiptDateTime', 'WorkstationID','POSItemID', 'RefundQuantity','RefundUnitAmount', 'RefundAmount'])
    for l1, Transaction in enumerate(root.findall('Transaction')):
        if l1==0:
            rw=l1
        else:
            rw=len(df)
        root1=Transaction
        seq = Transaction.find('SequenceNumber').text
        recDt = Transaction.find('ReceiptDateTime').text
        ws = Transaction.find('WorkstationID').text

        for l2, bu in enumerate(root.iter('UnitID')):
            bu = bu.text

        for l3, LineItem in enumerate(root.iter('LineItem')):
            root1=LineItem
            if l3!=0:
                rw = len(df)
            if(root1.findall('Return')):
                for l5, sale in enumerate(root1.findall('Return')):
                    Qnt = sale.find('Quantity').text
                    uAmt = sale.find('UnitRetailPrice').text
                    refAmt = sale.find('ExtendedAmount').text
                    df.at[rw,'RefundQuantity']=Qnt
                    df.at[rw,'RefundUnitAmount']=uAmt
                    df.at[rw,'RefundAmount']=refAmt
                    root2=sale
                    if(root2.findall('POSIdentity')):
                        for l6, pos in enumerate(root2.findall('POSIdentity')):
                            posItem = pos.find('POSItemID').text
                            df.at[rw,'POSItemID']=posItem
                    root2=sale
                    
            df.at[rw,'UnitID']=bu
            df.at[rw,'RefundSequenceNumber']=seq
            df.at[rw,'RefundReceiptDateTime']=recDt
            df.at[rw,'WorkstationID']=ws
    return df

In [134]:
def compare_klog_kcm(dfs,kcms,base_path_out):
    for order_no in list(dfs.keys()):
        klog_df = dfs[order_no]
        kcms_df = kcms[order_no]

        src_df = klog_df.copy()
        trg_df = kcms_df.copy()

        src_df = src_df[['Division','Store_no','POSItemID', 'NetQuantity', 'NetAmount']] #,'ReceiptDateTime'
        trg_df = trg_df[['BIL_DIV_NO','STO_NO','CON_UPC_NO','SCN_UNT_QY', 'SCN_DOL_AM','ROW_ADD_TS']]

        trg_df['CON_UPC_NO'] = trg_df['CON_UPC_NO'].apply(lambda x: '0'+str(x))

        df =src_df.merge(trg_df, how= 'left', left_on =['POSItemID'], right_on =['CON_UPC_NO'])

        df['match'] = [ 'True' if ((sq==tq) & (sa==ta)) else 'False' for sq,sa, tq, ta in zip(df['NetQuantity'],df['NetAmount'],df['SCN_UNT_QY'],df['SCN_DOL_AM'])]

        df['total_cnt'] = df.groupby('ROW_ADD_TS')['match'].transform('count')

        df1=df.groupby(['ROW_ADD_TS','total_cnt'])['match'].apply(lambda x: (x=='True').sum()).reset_index(name='count')

        df1=df1[df1['total_cnt']==len(src_df)].copy()

        upload_date_list = list(df1[df1['count']==df1['count'].max()]['ROW_ADD_TS'].astype(str))
        if len(upload_date_list)>0:
            upload_date = upload_date_list[0]
            trg_df=trg_df[trg_df['ROW_ADD_TS']==upload_date][['BIL_DIV_NO','STO_NO','CON_UPC_NO','SCN_UNT_QY', 'SCN_DOL_AM']].copy()
        
        trg_df = trg_df[['BIL_DIV_NO','STO_NO','CON_UPC_NO','SCN_UNT_QY', 'SCN_DOL_AM']]
        trg_df.columns= src_df.columns

        src_df.set_index('POSItemID', inplace=True)
        trg_df.set_index('POSItemID', inplace=True)

        src_df.sort_index(inplace=True)
        trg_df.sort_index(inplace=True)

        comparison = pvd.core.compare(src_df,trg_df,"demo")
        
#         comparision.apply_style().to_excel(outputfile)
        strformat = '%m%d%Y' + 'T' + '%H%M%S'
        filename = f'KCMS_Comparison_Report_{order_no}_' + dt.datetime.today().strftime(strformat) + '.xlsx'
        output_file = os.path.join(base_path_out,filename)
        comparison.apply_style().to_excel(output_file)

In [135]:
def compare_klog_kods(dfs,kdfs,base_path_out):
    for order_no in list(dfs.keys()):
        klog_df = dfs[order_no]
        kods_df = kdfs[order_no]

        src_df = klog_df.copy()
        trg_df = kods_df.copy()

        src_df = src_df[['Division','Store_no','POSItemID', 'NetQuantity', 'NetAmount']] #,'ReceiptDateTime'
        trg_df = trg_df[['MGT_DIV_NO','STO_NO','GTIN_NO','SCN_UNT_QY', 'RTL_DOL_AM']]

        trg_df['STO_NO'] = trg_df['STO_NO'].apply(lambda x: (str(x).strip()).zfill(5))

        trg_df.columns= src_df.columns

        src_df.set_index('POSItemID', inplace=True)
        trg_df.set_index('POSItemID', inplace=True)

        src_df.sort_index(inplace=True)
        trg_df.sort_index(inplace=True)

        comparison = pvd.core.compare(src_df,trg_df,"demo")
        
        strformat = '%m%d%Y' + 'T' + '%H%M%S'
        filename = f'KODS_Comparison_Report_{order_no}_' + dt.datetime.today().strftime(strformat) + '.xlsx'
        output_file = os.path.join(base_path_out,filename)
        comparison.apply_style().to_excel(output_file)

# Main Code Starts from here

In [136]:
# list all the files under provided folder path
xmlfiles =[x for x in os.listdir(folder_path) if str(x).endswith('.xml')]

xmlfiles

['1680096201905.xml']

In [137]:
# {orderno1:{ 'salesxml' : f1; 'refundxml' : [f2, f3, f4]}}

In [138]:
ordergroupdict = get_order_group(xmlfiles)
ordergroupdict

{'1680096201905': '1680096201905.xml'}

In [139]:
ordergroupdict = {'1680096201905':'1680096201905.xml'}

In [140]:
col=['Division','Store_no','UnitID', 'SequenceNumber', 'ReceiptDateTime', 'WorkstationID','POSItemID', 'Quantity','UnitAmount', 'ExtendedAmount','SubtractAmt','NetQuantity','NetAmount','Desc']

tlog_queries= {}
kcms_queries={}
pdict = {}
dfs = {}

# Processing each Order No one by one
for k,v in ordergroupdict.items():
    print(f"######################Order no : {k}######################")
    xml_list =  v #.get('refundxml','FILENOTFOUND')
    seq = []
    div = store = wid = gtin = qdate = qtime = ''
    fname = v #.get('salesxml','FILENOTFOUND')
    if fname=='FILENOTFOUND':
        print(f"Sales Xml file not found for Order no {k}")
        continue
    
    #Processing main xml files for Current Order no
    
    print(f"processing Main xml File {fname}")
    new_xml_file = processesing(fname,folder_path)
    tree=et.parse(new_xml_file)
    root=tree.getroot()
    xml_df = parse_xml_to_df(root)
    xml_df =xml_df.dropna(subset=['ExtendedAmount','POSItemID','Quantity'], how='all').reset_index(drop=True)

    xml_df['Desc']=[f"({q}, {a:.2f})" if (q==q and a==a) else pd.NA for q,a in zip(xml_df['NetQuantity'],xml_df['NetAmount'])]        
#         print(xml_df)
    xml_df['Division']=str(xml_df['UnitID'][0])[:3]
    xml_df['Store_no']=str(xml_df['UnitID'][0])[3:]
    # Storing all the required values to pass it further to sql query
    div = str(xml_df['UnitID'][0])[:3]
    store = str(xml_df['UnitID'][0])[3:]
    wid = str(xml_df['WorkstationID'][0])
    gtins = ",".join(list(set(xml_df['POSItemID'])))
    seq_list = [x for x in list(set(xml_df['SequenceNumber'])) if x==x]
    seq = list(set(seq_list))
    qdatetime = list(set(xml_df['ReceiptDateTime']))
    qdate, qtime = str(xml_df['ReceiptDateTime'][0]).split('T')
    
#     for refund_file in xml_list:
#         fname = refund_file
#         print(f"processing Refund xml File {fname}")
#         new_xml_file = processesing(fname,folder_path)
#         tree=et.parse(new_xml_file)
#         root=tree.getroot()
#         refund_xml_df = parse_refundxml_to_df(root)
#         refund_xml_df =refund_xml_df.dropna(subset=['RefundAmount','POSItemID','RefundQuantity'], how='all').reset_index(drop=True)
        
#         refund_xml_df['RefundAmount'] = refund_xml_df['RefundAmount'].fillna(0).astype(float)
#         refund_xml_df['RefundQuantity'] = refund_xml_df['RefundQuantity'].fillna(0).astype(int)
        
#         xml_df = xml_df.merge(refund_xml_df, how='left',on=['UnitID', 'WorkstationID','POSItemID'])
#         seq_list = [x for x in list(set(xml_df['RefundSequenceNumber'])) if x==x]
#         seq = [*seq,*seq_list]
#         rqdatetime = [x for x in list(set(xml_df['RefundReceiptDateTime'])) if x==x]
#         qdatetime = [*qdatetime,*rqdatetime]
#         xml_df['Desc']=[", ".join([d,f"({q}, {a:.2f})"])  if (q==q and a==a) else d for d,q,a in zip(xml_df['Desc'],xml_df['RefundQuantity'],xml_df['RefundAmount'])]
#         xml_df['NetAmount']=xml_df['NetAmount'].sub(xml_df['RefundAmount'], fill_value=0)
#         xml_df['NetQuantity']=xml_df['NetQuantity'].sub(xml_df['RefundQuantity'], fill_value=0)
        
#         print('-------------------------------------------------------------------')
#         print(xml_df[['POSItemID', 'Quantity','ReceiptDateTime', 'RefundReceiptDateTime','ExtendedAmount','NetQuantity','NetAmount','Desc']])
#         print('-------------------------------------------------------------------')
#         xml_df=xml_df[col]
   
    xml_df.sort_values(by='POSItemID',inplace=True)
    xml_df['NetAmount']=xml_df['NetAmount'].astype(float).round(2)
    xml_df['NetQuantity']=xml_df['NetQuantity'].astype(int)
    print('**********************************************************************************')
    print(xml_df[['Division','Store_no','POSItemID', 'Quantity','ReceiptDateTime', 'ExtendedAmount','NetQuantity','NetAmount','Desc']])
    print('**********************************************************************************')
    dfs[k] = xml_df[['Division','Store_no','ReceiptDateTime','POSItemID', 'Quantity', 'ExtendedAmount','SubtractAmt','NetQuantity','NetAmount','Desc']]
    seq_list = [x for x in list(set(seq)) if x==x]
    sequence = ", ".join(seq_list)
    query = f"""
SELECT MGT_DIV_NO ,STO_NO ,FAC_ID ,TRN_DT ,TRN_TM ,TML_NO ,TRN_NO ,ITM_ID ,GTIN_NO ,SCN_UNT_QY ,GTIN_UOM_AM ,CLY_DOL_AM ,RTL_DOL_AM
from tlog.vw_fct_itm 
where
MGT_DIV_NO in ({div})
AND STO_NO = {store}
AND TML_NO = {wid}
AND TRN_NO IN ({sequence})
AND TRN_DT =  '{qdate}'
AND GTIN_NO IN ({gtins})
AND TRN_TM IN ('{qtime}')

with ur for read only
"""
    kcms_query = f"""
with bill_id as 
( select FAC_BIL_DIV_ID from kcms.dim_fac_lvl_fac where FAC_NO = '{store}' 
and FAC_BIL_DIV_NO = '{div}'
)

SELECT d.day_dt,st.bil_div_no,st.sto_no,c.CON_UPC_NO,scn_dol_am,scn_unt_qy,scn_lbs_am,ROW_ADD_TS
FROM kcms.stg_sls_day_sto_con_rms_{div} f
INNER JOIN kcms.dim_pdt_lvl_con c ON f.CON_UPC_ID = c.CON_UPC_ID
INNER JOIN kcms.dim_cal_lvl_day d ON d.day_id = f.day_id
INNER JOIN kcms.dim_Sto_lvl_sto st ON st.sto_id = f.sto_id
INNER JOIN bill_id b ON f.BIL_DIV_ID = b.FAC_BIL_DIV_ID and c.bil_div_id = b.FAC_BIL_DIV_ID
WHERE d.day_dt = '{qdate}'
and d.BGN_WEK_DAY_ID = 1
AND c.CON_UPC_NO IN ({gtins});
"""
    
    tmpdict={}
    tmpdict={'div':div,'store':store,'wid':wid,'sequence':sequence,'qdate':qdate,'qtime':qtime,'gtins':gtins,'qdatetime':qdatetime}
    pdict[k] =tmpdict
    tlog_queries[k]=query
    kcms_queries[k]=kcms_query
    
#     kcms_df = pd.read_sql_query(kcms_query, kcmst_conn)
#     kcms_df.sort_values(by='CON_UPC_NO',inplace=True)
#     kdfs[i] = kcms_df
    
    


######################Order no : 1680096201905######################
processing Main xml File 1680096201905.xml
**********************************************************************************
  Division Store_no       POSItemID Quantity      ReceiptDateTime  \
6      014    00007  00001901461090        1  2023-03-29T09:18:12   
5      014    00007  00002420003885        1  2023-03-29T09:18:12   
7      014    00007  00003040021118        1  2023-03-29T09:18:12   
2      014    00007  00003320000322        1  2023-03-29T09:18:12   
3      014    00007  00003320097551        1  2023-03-29T09:18:12   
0      014    00007  00004200044364        1  2023-03-29T09:18:12   
4      014    00007  00004200044364        1  2023-03-29T09:18:12   
1      014    00007  00007261373915        1  2023-03-29T09:18:12   
8      014    00007  00010000008313        1  2023-03-29T09:18:12   
9      014    00007  00010000008664        1  2023-03-29T09:18:12   

  ExtendedAmount  NetQuantity  NetAmount     

In [141]:
# sales xml will not have any refund and there will be only one sales xml
# all other xml will check for Refund data 

In [142]:
df1=dfs['1680096201905']

In [143]:
df1.sort_values(by=['POSItemID'])

,Division,Store_no,ReceiptDateTime,POSItemID,Quantity,ExtendedAmount,SubtractAmt,NetQuantity,NetAmount,Desc
6,014,00007,2023-03-29T09:18:12,00001901461090,1,26.99,2.8,1,24.19,"(1.0, 24.19)"
5,014,00007,2023-03-29T09:18:12,00002420003885,1,16.99,4.0,1,12.99,"(1.0, 12.99)"
7,014,00007,2023-03-29T09:18:12,00003040021118,1,6.99,1.0,1,5.99,"(1.0, 5.99)"
2,014,00007,2023-03-29T09:18:12,00003320000322,1,13.99,3.0,1,10.99,"(1.0, 10.99)"
3,014,00007,2023-03-29T09:18:12,00003320097551,1,8.99,8.99,1,0.00,"(1.0, 0.00)"
0,014,00007,2023-03-29T09:18:12,00004200044364,1,27.99,5.0,1,22.99,"(1.0, 22.99)"
4,014,00007,2023-03-29T09:18:12,00004200044364,1,27.99,5.0,1,22.99,"(1.0, 22.99)"
1,014,00007,2023-03-29T09:18:12,00007261373915,1,9.99,9.99,1,0.00,"(1.0, 0.00)"
8,014,00007,2023-03-29T09:18:12,00010000008313,1,1.0,NaN,1,1.00,"(1.0, 1.00)"
9,014,00007,2023-03-29T09:18:12,00010000008664,1,1.0,NaN,1,1.00,"(1.0, 1.00)"


In [144]:
5.0 +1.0 +0.1 +2.7 +1.0 +3.0 +5.0 +8.99 +3.0 +9.99 +0.0  

39.78

In [145]:
df1['ExtendedAmount'].astype(float).sum()

141.92

In [146]:
df1['SubtractAmt'].astype(float).sum()

39.78

In [119]:
139.92-41.78

98.13999999999999

In [121]:
139.92-41.78 + 6.76 +15.1

(total amt - coupon gtin amount) - (total subtract amount+ coupon gtin amount) + tax + tender change

119.99999999999999

In [ ]:
Total extended amount only typecode 00/01 -sum of subtract - sum of gtin coupons + tax

In [113]:
df1['NetAmount'].astype(float).sum()

102.13999999999999

In [116]:
102.13999999999999-2+6.76

106.89999999999999

In [90]:
141.92-39.78+6.76-2

139.92 - 41.78

139.92+6.76

146.67999999999998

In [87]:
139.92+6.76
GROSS POSITIVE =  TOTAL EXTENDED AMOUNT + TAX AMOUNT
GROSS NEGATIVE = TOTAL SUBTRACT AMOUNT + EXTRA COUPON AMOUNT (POSIDType=75)

146.67999999999998

In [88]:
146.68-41.78

104.9

In [82]:
112.84+6.76 -2

117.60000000000001

In [81]:
5.0 +1.0 +0.1 +2.7 +1.0 +3.0 +5.0 +8.99 +3.0 +9.99 +0.0  

39.78

# KCMS Comparison

In [40]:
kcms = {}
for i, query in kcms_queries.items():
    with open("kcmsquery.sql",'w+') as f:
        f.write(query)
    df = pd.read_sql_query(query, kcmst_conn)
    df.sort_values(by='CON_UPC_NO',inplace=True)
    df['SCN_DOL_AM']=df['SCN_DOL_AM'].astype(float).round(2)
    df['SCN_UNT_QY']=df['SCN_UNT_QY'].astype(int)
    kcms[i] = df
    print(df)

Empty DataFrame
Columns: [DAY_DT, BIL_DIV_NO, STO_NO, CON_UPC_NO, SCN_DOL_AM, SCN_UNT_QY, SCN_LBS_AM, ROW_ADD_TS]
Index: []


In [36]:
compare_klog_kcm(dfs,kcms,folder_path)

KeyError: 'TARGET'

# KODS TLOG Comparison

In [147]:
kdfs = {}
seq_dict = {}
for i, query in tlog_queries.items():
    with open("kodsquery.sql",'w+') as f:
        f.write(query)
    df = pd.read_sql_query(query, kods_conn)
    df.sort_values(by='GTIN_NO',inplace=True)
    df['RTL_DOL_AM']=df['RTL_DOL_AM'].astype(float).round(2)
    df['SCN_UNT_QY']=df['SCN_UNT_QY'].astype(int)
    seq_dict[i] = list(df['TRN_NO'])
    kdfs[i] = df
    print(df)

Empty DataFrame
Columns: [MGT_DIV_NO, STO_NO, FAC_ID, TRN_DT, TRN_TM, TML_NO, TRN_NO, ITM_ID, GTIN_NO, SCN_UNT_QY, GTIN_UOM_AM, CLY_DOL_AM, RTL_DOL_AM]
Index: []


In [38]:
compare_klog_kods(dfs,kdfs,folder_path)

KeyError: 'TARGET'

# Missing Sequence no from KODS TLOG

In [66]:
missing_sequence_no={}
for k,v in pdict.items():
    klog=v['sequence'].split(',')
    tlog = seq_dict[k]
    missing_sequence_no[k]=[x for x in klog if x not in tlog]

In [67]:
missing_sequence_no

{'1230045299981171201': ['7', ' 7424', ' 7566', ' 7563'],
 '1230046319662747301': ['8', ' 5539']}